In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

## Getting enablement data

In [3]:
# XGS qualifications count at wire center level - manually update date filters every month and at the beginning of the year

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_ECD >= '1-DEC-2022' and O.UNI_ECD < '1-JAN-2023' then 'Y' end as MONTH_CNT_YN
                    ,case when O.UNI_ECD >= '1-JAN-2022' and O.UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_xgs_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_xgs_enabled_cnt
                    ,count(distinct LUID) as cumulative_xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd >= '1-JAN-2022' --'1-JAN-2023' 
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
%%time

# count of MG enabled LUs at wire center level - manually update date filters
# 10/10/22 - update multigig enablement date to reflect mg turn up date

mg_query = """
           
           WITH MG_DATES AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_POLYGONID in ('1179498','1179547','1179121','1179499') then to_date('30-SEP-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179687','1179674') then to_date('10-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179569','1179553','1179688') then to_date('14-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179864','1180012') then to_date('15-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179808','1179862','1179836','1179604','1179838','1179887',
                                                   '1180054','1180222','1179157','1179574') then to_date('21-OCT-2022','DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180028','1180254','1179583') then to_date('27-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179976') then to_date('28-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179774','1180265','1180107','1180306','1180026') then to_date('4-NOV-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180288') then to_date('7-NOV-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180067','1179849','1180344','1179924') then to_date('10-NOV-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179708','1179714','1179719','1180072','1180075','1180089','1180095','1180096',
                                                   '1180391','1179639','1179689','1179690','1179700','1179701','1179702','1179703',
                                                   '1179704','1179718','1179793','1179803','1180092','550715','1180237','1180315',
                                                   '1180031') then to_date('18-NOV-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180094','1180507','1180494','1179641',
                                                   '1179914','1189074') then to_date('22-NOV-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180076','1180079','1180492','1180497','1180333',
                                                   '1179840','1180355','1180086','1180111','1180327'
                                                    ) then to_date('3-DEC-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179913','1180091','1179576','1179672','1179971','1179977',
                                                   '1179981','1179987','1179988','1179989','1179994','1180005','1180006',
                                                   '1180041','1180123','1189130','1179352','1179858','1180015','1180098',
                                                   '1180392','1180399'           
                                                    ) then to_date('9-DEC-2022', 'DD-MON-YYYY')                                 
                         when O.UNI_POLYGONID in ('1180486','1180027','1179922','1180002','1180468','1179979'          
                                                    ) then to_date('16-DEC-2022', 'DD-MON-YYYY')
                         when O.UNI_POLYGONID in ('1180491','1180509','1189253','1189283','1180488',
                                                  '1189200','1189088','1189058','1180436','1180496'
                                                    ) then to_date('22-DEC-2022', 'DD-MON-YYYY')
                          
                          else O.UNI_ECD end
                          AS UNI_ECD
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                    
                                       '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL    
                                          ) --add in MG polygon IDs here
                    
            ),
            QD AS (
            
            SELECT  LUID
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,ADDRESS
                    ,UNI_SERVICETYPE
                    ,OLT
                    ,TECH_TYPE
                    ,OLT_TYPE
                    ,FSAI
                    ,UNI_POLYGONID
                    ,UNI_STATUS
                    ,UNI_NDSJOBNUMBER
                    ,UNI_FWRKSJOBNUMBER
                    ,case when UNI_ECD >= '1-DEC-2022' and UNI_ECD < '1-JAN-2023' then 'Y' end as MONTH_CNT_YN
                    ,case when UNI_ECD >= '1-JAN-2022' and UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM MG_DATES
            
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_mg_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_mg_enabled_cnt
                    ,count(distinct LUID) as cumulative_mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 724 ms


In [5]:
mg_lu_ora.groupby(['STATE'], as_index = False).sum()

,STATE,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,4156,14316,14316
1,CO,3709,4920,4920
2,MN,1358,9711,9711
3,OR,85,349,349
4,UT,419,659,659
5,WA,1740,2080,2080


In [6]:
# sum queries at wire center level - add mg flag according to mg enablement or core network status
xgs_enablement_wc = df_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()
mg_enablement_wc = mg_lu_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()

In [7]:
# merge xgs and mg enablement
luqual_enablement_df = xgs_enablement_wc.merge(mg_enablement_wc, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #save copy

In [8]:
luqual_enablement_df

,STATE,PRIMARY_WIRE_CENTER_ID,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,PHNXAZNE,0,210,210,0.0,210.0,210.0
1,AZ,PHNXAZNO,0,243,243,243.0,243.0,243.0
2,AZ,PHNXAZSO,299,1624,1624,299.0,1624.0,1624.0
3,AZ,PHNXAZWE,0,195,195,0.0,0.0,0.0
4,AZ,SPRSAZEA,0,296,296,0.0,0.0,0.0
5,AZ,SPRSAZMA,0,259,259,0.0,0.0,0.0
6,AZ,TCSNAZCR,0,400,400,0.0,400.0,400.0
7,AZ,TCSNAZFW,0,601,601,0.0,0.0,0.0
8,AZ,TCSNAZMA,1219,11522,11522,3053.0,11278.0,11278.0
9,AZ,TCSNAZRN,0,1192,1192,561.0,561.0,561.0


In [9]:
# check data
scorecard_view = luqual_enablement_df[['STATE','CUMULATIVE_XGS_ENABLED_CNT','CUMULATIVE_MG_ENABLED_CNT','MONTHLY_XGS_ENABLED_CNT','MONTHLY_MG_ENABLED_CNT']].groupby(['STATE'], as_index = False).sum()

In [10]:
# import salesforce summary to get active subs
sf_df_xgs_sum = pd.read_excel(r'SalesforceReportExport/Salesforce_Summary.xlsx',sheet_name = 'State_Summary', skiprows=[0,1])
sf_df_mg_sum = pd.read_excel(r'SalesforceReportExport/Salesforce_Summary.xlsx',sheet_name = 'MultiGig_State_Summary', skiprows=[0,1]) 

In [11]:
sf_df_xgs_sum = sf_df_xgs_sum[['STATE','Total Active Subscribers']].rename(columns = {'Total Active Subscribers':'Total_XGS_Active_subs'})
sf_df_mg_sum = sf_df_mg_sum[['STATE','Total Active Subscribers']].rename(columns = {'Total Active Subscribers':'Total_MG_Active_subs'})
salesforce_subscribers_summary = sf_df_xgs_sum.merge(sf_df_mg_sum, how = 'left', on = 'STATE')
salesforce_subscribers_summary = salesforce_subscribers_summary.loc[salesforce_subscribers_summary['STATE']!=0,:]

In [12]:
scorecard_view = scorecard_view.merge(salesforce_subscribers_summary, on = 'STATE', how = 'left')

# check states in AZ, CO, FL, MN, OR, UT, WA, IA, NV otherwise move grand total position in templates:
scorecard_view

,STATE,CUMULATIVE_XGS_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT,MONTHLY_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,Total_XGS_Active_subs,Total_MG_Active_subs
0,AZ,16987,14316.0,1518,4156.0,287,70.0
1,CO,9276,4920.0,662,3709.0,162,36.0
2,FL,27539,0.0,1662,0.0,200,NaN
3,MN,10508,9711.0,1333,1358.0,159,64.0
4,NV,94,0.0,94,0.0,1,NaN
5,OR,349,349.0,0,85.0,1,1.0
6,UT,4060,659.0,124,419.0,62,1.0
7,WA,2450,2080.0,159,1740.0,115,29.0


## Fill Tempate (Import enablement view finalized at this step)

In [13]:
import_template = pd.read_excel("Template/2022Dec_PathToPremier_template.xlsx", 
                                sheet_name="Sheet1",
                                skiprows=[0])

# replace scorecard monthly XGS enabled column
import_xgs_enablement = pd.read_excel(
                                "Template/Enablement.xlsx",
                                sheet_name="V2",
                                usecols=[0,8])

import_xgs_enablement = import_xgs_enablement.rename(
                                columns={'State':'STATE',
                                         'Month_enabled':'MONTHLY_XGS_ENABLED_CNT'})
import_xgs_enablement = import_xgs_enablement[import_xgs_enablement['STATE']!='Grand Total']

# fill one with another
df1 = import_xgs_enablement.set_index("STATE")
df2 = scorecard_view.set_index("STATE")
df2.update(df1)
df2 = df2.reset_index()
scorecard_view = df2

state_base = import_template.rename(columns={'Market':'STATE'})
state_base = state_base[state_base['STATE']!="TOTAL"]

# has correct ordered states
scorecard_addition_view = state_base[["STATE"]].merge(scorecard_view,
                on='STATE',
                how='outer').fillna(0)

scorecard_addition_view

,STATE,CUMULATIVE_XGS_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT,MONTHLY_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,Total_XGS_Active_subs,Total_MG_Active_subs
0,WA,2450.0,2080.0,164.0,1740.0,115.0,29.0
1,MN,10508.0,9711.0,1709.0,1358.0,159.0,64.0
2,CO,9276.0,4920.0,1030.0,3709.0,162.0,36.0
3,AZ,16987.0,14316.0,3379.0,4156.0,287.0,70.0
4,OR,349.0,349.0,0.0,85.0,1.0,1.0
5,UT,4060.0,659.0,713.0,419.0,62.0,1.0
6,FL,27539.0,0.0,4536.0,0.0,200.0,0.0
7,IA,0.0,0.0,0.0,0.0,0.0,0.0
8,NV,94.0,0.0,94.0,0.0,1.0,0.0


In [14]:
# fill enablement data
with pd.ExcelWriter('Template/2022Dec_PathToPremier_template.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    scorecard_addition_view.iloc[:,1:-2].to_excel(writer, 
                      sheet_name="Sheet1", 
                      startrow=2,
                      startcol=5,
                      index=False, 
                      header=False) 
    scorecard_addition_view.iloc[:,-2:].to_excel(writer, 
                      sheet_name="Sheet1", 
                      startrow=2,
                      startcol=11,
                      index=False, 
                      header=False) 

In [15]:
# create archived copy
import shutil
import time

TodaysDate = time.strftime("%m%d%y")

excelfilename = "Template/archive/" + TodaysDate +"_PathToPremier_View.xlsx"
shutil.copyfile('Template/2022Dec_PathToPremier_template.xlsx', excelfilename)

'Template/archive/122122_PathToPremier_View.xlsx'